In [1]:
from sklearn.preprocessing import FunctionTransformer

from reskit.norms import binar_norm

from reskit.core import Transformer, Pipeliner

from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import os
import pandas as pd
import numpy as np

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def orig(x):
    return x

import matrix_eig as me

from matplotlib import pyplot as plt
%matplotlib inline
import re

In [2]:
def get_autism(path_to_read='../Data/dti/', distances=True):
    def get_autism_distances(loc_name):
        with open(loc_name, 'r') as f:
            read_data = f.readlines()

        read_data = pd.DataFrame(
            np.array([np.array(item[:-1].split()).astype(int) for item in read_data]))

        return read_data

    def get_distance_matrix(coords):
        if type(coords) == pd.core.frame.DataFrame:
            coords = coords.values
        elif type(coords) != np.ndarray:
            print('Provide either pandas df or numpy array!')
            return -1

        shape = len(coords)
        dist_matrix = np.zeros((shape, shape))
        del shape
        for i in range(len(coords)):
            for j in range(i + 1, len(coords)):
                dist_matrix[i, j] = np.linalg.norm(coords[i, :] - coords[j, :])
                dist_matrix[j, i] = dist_matrix[i, j]
        return dist_matrix

    target_vector = []  # this will be a target vector (diagnosis)
    matrices = []  # this will be a list of connectomes
    all_files = sorted(os.listdir(path_to_read))
    matrix_files = [
        item for item in all_files if 'DTI_connectivity' in item and 'All' not in item]
    distance_files = [
        item for item in all_files if 'DTI_region_xyz_centers' in item and 'All' not in item]

    # for each file in a sorted (!) list of files:
    for filename in matrix_files:

        A_dataframe = pd.read_csv(
            path_to_read + filename, sep='   ', header=None, engine='python')
        A = A_dataframe.values  # we will use a list of numpy arrays, NOT pandas dataframes
        matrices.append(A)# append a matrix to our list
        if "ASD" in filename:
            target_vector.append(1)
        elif "TD" in filename:
            target_vector.append(0)
    asd_dict = {}
    asd_dict['X'] = np.array(matrices)
    asd_dict['y'] = np.array(target_vector)
    if distances:
        dist_matrix_list = []
        for item in distance_files:
            # print(item)
            cur_coord = get_autism_distances(path_to_read + item)
            cur_dist_mtx = get_distance_matrix(cur_coord)
            dist_matrix_list += [cur_dist_mtx]

        asd_dict['dist'] = np.array(dist_matrix_list)

    return asd_dict

In [3]:
params = {
    'text.usetex'         : True,
    'text.latex.unicode'  : True,
    'text.latex.preamble' : r"\usepackage[T2A]{fontenc}",
    'font.size'           : 15,
    'font.family'         : 'lmodern'
    }

plt.rcParams.update(params)



def print_boxplot(file_name, mod, figsize = (10.5,6.5)):
    ##################################################################
    # Paramatrs:                                                     #
    #----------------------------------------------------------------#
    # file_name: string, name of file witch has a table with columns:# 
    #           'eval_roc_auc_scores' and 'Featurizers'              #
    #                                                                #
    # mod: string, additional information for boxplot                #
    #                                                                #
    # figsize: w,h tuple in inches                                   #
    ##################################################################
    data = []
    result = pd.read_csv(file_name, index_col=0)
    
    lables          = result['Featurizers']
    array_of_arrays = result['eval_roc_auc_scores']
    for tmp in array_of_arrays:
        array_str = re.split('[]*[ ,]', tmp)
        ##########################################
        #Костыль
        for i in range(array_str.count('')):
            array_str.remove('')
        ##########################################
        array_fold = []
        for i in array_str:
            array_fold.append(float(i))
        data.append(array_fold)
        
    fig = plt.figure(figsize=figsize)
    ax  = fig.add_subplot(111)
    bp  = ax.boxplot(data, 0, '+', labels=lables)
    
    plt.setp(bp['boxes'],    color='DarkGreen')
    plt.setp(bp['whiskers'], color='DarkOrange', linestyle = '-')
    plt.setp(bp['medians'],  color='DarkBlue')
    plt.setp(bp['caps'],     color='Gray')
    
    
    ax.set_title(r'ROC AUC mean for LogisticRegression (' + mod + ').' )
    ax.set_ylabel(r'ROC AUC mean')
    ax.set_xlabel(r'k')
    ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',alpha=0.5)
    
    plt.show()

## Train LogRegression and conctrsuct BoxPlot

In [ ]:
def add_ind(ind):
    ind_new = []
    for i in ind:
        for j in range(7):
            ind_new.append(i+j)
    return ind_new


class my_eval:
    
    def __init__(self, eval_cv):
        self.eval = eval_cv
        
    def split(self, X, y, groups):
        ind = np.arange(X.shape[0] / 7) * 7
        y_min = y[ind]
        
        for train_index, test_index in self.eval.split(ind, y_min):
            ind_train, ind_test = ind[train_index], ind[test_index]
            yield add_ind(ind_train), add_ind(ind_test)

In [4]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = my_eval(StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1))

In [5]:
#1

In [ ]:
data = [('UCLAsource', Transformer(get_autism))]

weighters = [('binar', Transformer(binar_norm))]

normalizers = [('origN', Transformer(orig))]

featurizers = [('Orig',   Transformer(me.orig_vec,       collect=['X_vec'])),
               ('0',      Transformer(me.matrix_eig_0,   collect=['X_vec'])),
               ('20',     Transformer(me.matrix_eig_20,  collect=['X_vec'])),
               ('40',     Transformer(me.matrix_eig_40,  collect=['X_vec'])),
               ('60',     Transformer(me.matrix_eig_60,  collect=['X_vec'])),
               ('80',     Transformer(me.matrix_eig_80,  collect=['X_vec'])),
               ('100',    Transformer(me.matrix_eig_100, collect=['X_vec'])),
               ('120',    Transformer(me.matrix_eig_120, collect=['X_vec'])),
               ('140',    Transformer(me.matrix_eig_140, collect=['X_vec'])),
               ('160',    Transformer(me.matrix_eig_160, collect=['X_vec'])),
               ('180',    Transformer(me.matrix_eig_180, collect=['X_vec'])),
               ('200',    Transformer(me.matrix_eig_200, collect=['X_vec'])),
               ('220',    Transformer(me.matrix_eig_220, collect=['X_vec'])),
               ('240',    Transformer(me.matrix_eig_240, collect=['X_vec'])),
               ('260',    Transformer(me.matrix_eig_260, collect=['X_vec']))]


selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler())]

classifiers = [('LR', LogisticRegression())]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

param_grid = dict(
     LR=dict(
        C=[0.01] + [0.1*i for i in range(1, 11)],
        max_iter=[50, 100],
        penalty=['l1']
    )
    )

banned_combos = []

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,binar,origN,Orig,var_threshold,minmax,LR
1,UCLAsource,binar,origN,0,var_threshold,minmax,LR
2,UCLAsource,binar,origN,20,var_threshold,minmax,LR
3,UCLAsource,binar,origN,40,var_threshold,minmax,LR
4,UCLAsource,binar,origN,60,var_threshold,minmax,LR
5,UCLAsource,binar,origN,80,var_threshold,minmax,LR
6,UCLAsource,binar,origN,100,var_threshold,minmax,LR
7,UCLAsource,binar,origN,120,var_threshold,minmax,LR
8,UCLAsource,binar,origN,140,var_threshold,minmax,LR
9,UCLAsource,binar,origN,160,var_threshold,minmax,LR


In [ ]:
pipe.get_results('../Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'],
                 scoring=['roc_auc'], collect_n = 50, results_file = 'LR/binar_eig_rand_m.csv')

No previous results found.
Line: 1/15


In [ ]:
print_boxplot('LR/orig_eig_noabs_m.csv', 'Binar-Orig-MinMax-Rand')